In [1]:
#Use beautiful soup to retrive all MTA files from website (web scrape) just for 2020-01-01 to most recent
import re
import requests
import urllib.request
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

url = 'http://web.mta.info/developers/turnstile.html'

#https://blog.proxypage.io/web-scrape-using-python-in-less-than-5-minutes-2/
#https://hackersandslackers.com/scraping-urls-with-beautifulsoup/
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

#print(soup.prettify()) you can see 'soup' is our entire html page

URL_PREFACE = 'http://web.mta.info/developers/'
a = soup.find_all('a')
o = ()
for i in range(37,42):
#for i in range(37, len(a)):
    ins = (a[i].get_text(), URL_PREFACE + a[i]['href'],)
    o = o + (ins,)

col = ['C/A','UNIT','SCP','STATION','LINENAME','DIVISION','DATE','TIME','DESC','ENTRIES','EXITS']

data = pd.DataFrame()
for a, b in o:
    d = a[-4:]
    if d.find('2020') !=-1:
        df = pd.read_csv(b,  sep=r'\s*,\s*', header=0, engine='python') #else "EXITS" gives an error, python engine will avoid warning
        df['dt'] = df['DATE'] + ' ' + df['TIME']
        data = pd.concat([data, df])

print(data.DATE.min())        

data['entry_per_hour'] = data.ENTRIES - data.ENTRIES.shift(1)
#df['hourly_entry'] = hourly_entry.fillna(0)

data['exit_per_hour'] = data.EXITS - data.EXITS.shift(1)
#df['hourly_exit'] = hourly_exit.fillna(0)

data['use_per_hour'] = data.entry_per_hour + data.exit_per_hour

data['day'] = pd.to_datetime(data['dt']).dt.day_name()

data['hour'] = (pd.to_datetime(data['dt'])).dt.hour

11/28/2020


#### Cleaning data for extreme values

#### evidence of extremity

In [2]:
import matplotlib.pyplot as plt
#some graph


Point is: anything above __x__ is unrealistic, so we will ignore it.

In [3]:
#non-negative and LESS THAN 100000?
# clean 'HOURLY_ENTRIES'
#mask = data.entry_per_hour.astype(int) <= 20000
#data = data.loc(mask)

data['entry_per_hour'] = data.entry_per_hour.transform(
        lambda x: np.where((x<0)|(x>20000),x.mask((x<0)|(x>20000)).mean(),x))
    
# clean 'HOURLY_EXITS'
data['exit_per_hour'] = data.exit_per_hour.transform(
        lambda x: np.where((x<0)|(x>20000),x.mask((x<0)|(x>20000)).mean(),x))


data['use_per_hour'] = data.entry_per_hour + data.exit_per_hour


#### Proof that outliers are gone

In [4]:
#What graph can we show?

In [4]:
import datetime

def check_weekend(day):
    weekno = datetime.datetime.today().weekday()

    if weekno < 5:
        return False
    else:  # 5 Sat, 6 Sun
        return True

In [5]:
data.groupby(data['STATION']).sum()['use_per_hour']
stations = data['STATION'].unique()
print(len(stations))



days = data['day'].unique()
#data['weekend'] = check_weekend(data['day'])


378


In [6]:
import seaborn as sns
#data.groupby('DATE')['entry_per_hour','exit_per_hour'].sum().round(2).rename(
#    columns = {'DATE': "DATE", 'entry_per_hour': "TOTAL_ENTRIES",'exit_per_hour': 'TOTAL_EXITS'}).unstack().plot(ax=ax)


data = data.groupby('STATION')['use_per_hour'].sum().sort_values(ascending =False)
data.head(10)





STATION
34 ST-PENN STA     1.938145e+06
34 ST-HERALD SQ    1.537924e+06
GRD CNTRL-42 ST    1.443054e+06
86 ST              1.421662e+06
125 ST             1.335138e+06
42 ST-PORT AUTH    1.226611e+06
14 ST-UNION SQ     1.201797e+06
23 ST              1.138865e+06
59 ST              1.113387e+06
FLUSHING-MAIN      1.045703e+06
Name: use_per_hour, dtype: float64